# Uploading an image to the platform, and requesting a prediction for it
In this notebook, we will upload a single numpy image to a specific project on the platform, and make a request to get a prediction for it

In [ ]:
# As usual we will connnect to the platform first, using the credentials from the .env file

from dotenv import dotenv_values

from geti_sdk import Geti

env_variables = dotenv_values(dotenv_path=".env")

if not env_variables:
    print(
        "Unable to load login details from .env file, please make sure the file exists at the root of the notebooks directory."
    )

geti = Geti(
    host=env_variables.get("HOST"),
    username=env_variables.get("USERNAME"),
    password=env_variables.get("PASSWORD"),
)

### Setting up the project client, image client and prediction client
In this notebook, we will need three client objects: A ProjectClient to retrieve the project we want to upload to, an ImageClient to be able to upload the image and a PredictionClient to request and receive the prediction for the image. We first set up the ProjectClient, since we will need to get the project we are interested in before we can initialize the other two clients.

In [ ]:
from geti_sdk.rest_clients import ImageClient, PredictionClient, ProjectClient

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

Now that the project client is set up, we need to specify the project we would like to upload to. We will use the `COCO animal detection demo` that we created in notebook [002 create project from dataset](./002_create_project_from_dataset.ipynb).

In [ ]:
PROJECT_NAME = "COCO animal detection demo"

With the project name specified, we can retrieve the project details from the project client and use the returned `Project` object to set up an `image_client` and `prediction_client` for this project.

In [ ]:
project = project_client.get_project_by_name(PROJECT_NAME)
image_client = ImageClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)
prediction_client = PredictionClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

## Uploading an image
The 'data' folder in this directory contains an image 'dogs.png'. We can upload the image directly from file using the `image_client.upload_image()` method. Before uploading, we can get a list of all images in the project, so that we can verify that the image was uploaded successfully

In [ ]:
images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images.")

Now, we will upload the image

In [ ]:
image = image_client.upload_image(image="data/dogs.png")

Let's fetch the list of images again and see if it has changed

In [ ]:
images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images.")

Looks like the image has been added to the project, great! Let's use the prediction client to get a prediction for it

## Getting a prediction for the image
We will now try to request the prediction for the image. However, it is possible that the system is not able to return a prediction: This will happen when not all tasks in the project have a trained model to generate predictions from. In this case, the `prediction_client.get_image_prediction()` method will raise a ValueError. If a model has been trained for all trainable tasks in the project, the prediction client will return a `Prediction` object

> **NOTE**: Calling `prediction_client.get_image_prediction` for the first time may take a bit of time (several tens of seconds), because the system may have to start up its inference service. It could even cause the call to timeout, in that case you can just re-run the cell to try requesting the prediction again. Once the inference service is up and running, getting a prediction should be quick.

In [ ]:
from geti_sdk.demos import ensure_trained_example_project

# First, make sure that the project is trained and ready to predict
ensure_trained_example_project(geti=geti, project_name=project.name)

# Then, request the prediction
prediction = prediction_client.get_image_prediction(image)

## Visualizing the prediction
If the prediction succeeded, we can now have a closer look at the `prediction` object that was returned by the prediction client. We can either look at an overview of the data directly using `prediction.overview`, or we can visualize the prediction on the image using a helper function from the package. Both methods are shown in the cells below.

In [ ]:
print(prediction.overview)

In [ ]:
from geti_sdk.utils import show_image_with_annotation_scene

# To visualise the image, we have to retrieve the pixel data from the platform using the `image.get_data` method. The actual pixel data is
# downloaded and cached only on the first call to this method
image.get_data(geti.session)

show_image_with_annotation_scene(image, prediction, show_in_notebook=True)

## Uploading an image and getting a prediction -- The quick method
Because uploading and predicting an image is a pretty common operation, the `Geti` class provides a convenience method to do it in one go, without having to set up the ProjectClient, ImageClient and PredictionClient. Basically, this means you could do all of the above in one line of code! 

In [ ]:
quick_image, quick_prediction = geti.upload_and_predict_image(
    project_name=PROJECT_NAME,
    image="data/dogs.png",
    visualise_output=False,
    delete_after_prediction=False,
)
show_image_with_annotation_scene(quick_image, quick_prediction, show_in_notebook=True)